# Run ADAM 

The model ADAM (Annual Danish Aggregate Model)is a model of the Danish
economy maintained and developed by Danmarks Statistik.  A Link to the model: (https://www.dst.dk/pubomtale/18836). It has 4624 equations 

Danmarks Statistik has kindly provided the model version JUN17X and an associated databank for this demo. 

The model and databank is not the current model and forecast, but serves as an example. 

This is a basic demo. However you can use all the features of ModelFlow to manipulate the model.  

## Import ModelFlow and Pandas

In [ ]:
import pandas as pd

import sys
sys.path.append('modelflow/')

from modelsandbox import newmodel
from modelclass import model 

import  ipywidgets as widgets

from IPython.display import display, clear_output


## Read model and databank

In [ ]:
fadam = open('adam/jul17x.txt','rt').read()
bank  = pd.read_pickle('adam/lang100_2017.pc') 

## Make a model instance 

In [ ]:
turbo = 0     # if you set turbo=1 the model will be compiled, it takes time but specd the running up 
madam = newmodel(fadam,modelname='ADAM')    
madam.use_preorder = True  # for speedup 

## Run the Baseline 

In [ ]:
grund = madam(bank,2018,2030,conv=['YR','UL'],antal=100,alfa=0.4,ljit=turbo,chunk=30,relconv = 0.000000001,silent=True,debug=0)

## Prepare some stuff
The next cell defines variables and functions which makes an experiment and a simple interactive widget.

The cell is here for reference for the advanced user. 

You don't have to study the cell

In [ ]:
## make interactive widget
wvat  = widgets.FloatSlider(description='Value Added Tax rate',min=-5.,max=5.,value=-2.,
                            layout={'width':'50%'},style={'description_width':'30%'})
wuq  = widgets.FloatSlider(description='Extra labour supply, 1000 ',min=-10.,max=10.,value=0.,
                          layout={'width':'50%'},style={'description_width':'30%'})
wgo   = widgets.Button(description="Run")
w     = widgets.VBox([wvat,wuq,wgo])

# ADAM Variable descriptions 
trans = {'ENL'  :'Balance of Payment, surplus','UL':'Unemployment','FY':'Real GDP',
         'TFN_O':'Public surplus','TFN_OY':'Public surplus, Percent of GDP'}

# This function is run when the button is clecked 
def runadam(b):
    mulstart       = grund.copy()
    mulstart.TG    = grund.TG + wvat.value/100.
    mulstart.UQ    = grund.UQ - wuq.value
    mulstart.DUQSY = 1
   # with out:
    clear_output()
    mul = madam(mulstart,conv=['YR','UL'],antal=100,alfa=0.5,ljit=turbo,chunk=30,relconv = 0.000000001,silent=False,ldump=1)
    clear_output()
    display(w)
    _ =madam[['ENL','ul','fy','TFN_O','TFN_OY']].dif.rename(trans).plot(sharey=False,colrow=1,title='Response')

# Assign the function to the button  
wgo.on_click(runadam)
out = widgets.Output()

## Now make the experiment

In [ ]:
display(w)

## What drives GDP 
The graph can be small, but if you are using the jupyterlab version, you can inspect it through the PDF file. 

In [ ]:
_ = madam.FY.explain(up=1,pdf=1,dec=2)

## To small ?
The previous output can be difficult to read, as it can not be zoomed. 

However the output is also written to a .pdf file, which can be zoomed. The .pdf file is located at the graph/ subfolder. 
If you run this notebook in Windows the pdf=1 will trigger an automatic display of the .pdf file. 

If you are running this notebook on a virtual machine you can access the .pdf file yourself. By: 

 - go to the **Home** tab on the browser. 
 - Click on the graph/ folder. 
 - Either Click on the **FY explained.pdf** file, or if that don't work, download the file to your local machine.  
 

##  Uploade  and downloade to the virtual machine.
In general you should be able to both upload files from your local machine to the virtual machine, and and to download files from the virtual machine to the local machine.  

So even when though the virtual machines disappear you can still get the output to our local machine. And you can modify this notebook and download it for later use. 

## Now try turbo = 1 
Edit the cell above where turbo is set to 0 and change it to 1

Then run the notebook again. This will trigger a just in time compilation by the NUMBA library. The compilation will take time, but the simulation will be much faster.  